In [24]:
# Import necessary libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from warnings import filterwarnings

# Audio processing library
import librosa

# Scikit-learn for machine learning metrics
from sklearn import metrics

# Import necessary libraries for deep learning with tensorflow and Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import LeakyReLU, ReLU, PReLU, ELU
from tensorflow.keras.layers import Dropout

# Keras callback for early stopping
from keras.callbacks import EarlyStopping

# Set display options for Pandas DataFrame
pd.set_option('display.max_columns', 300)

# Ignore warnings for cleaner output
filterwarnings('ignore')

In [2]:
# Read the features and target datasets from CSV files into Pandas DataFrames

df_features = pd.read_csv('C:/Users/user/Desktop/Python/Main_Projects/Audio_Classification_Prj/features.csv')
df_target = pd.read_csv('C:/Users/user/Desktop/Python/Main_Projects/Audio_Classification_Prj/target.csv')

In [3]:
# Merge the features and target DataFrames based on a common column

df = pd.merge(df_features, df_target, how='left', on='Unnamed: 0')

In [5]:
df

,Unnamed: 0,0_x,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0_y
0,0,-217.35526,70.223380,-130.385270,-53.282898,-21.199130,-22.677622,-10.855970,18.294254,6.652703,14.324025,-12.167682,2.276837,-17.779188,10.388951,-6.582836,-0.694457,-18.336023,1.994253,-5.143332,8.302400,-12.645056,-6.529732,4.617668,-2.179917,-6.662824,0.359711,-3.908410,4.775624,-6.384521,-5.379818,0.915978,6.970494,-0.248668,1.678219,-5.611182,-2.964347,3.149057,-1.693053,-0.616984,0.386005,1.476512,0.913438,-3.818676,-5.332023,-4.797818,-0.537269,-4.900532,0.346234,-4.434357,1.368662,-0.527099,-3.310836,-0.774530,-0.343457,-4.175725,-3.187561,-1.243311,-3.111319,0.200477,-2.898059,1.379016,2.010003,-0.557578,1.518796,-1.570530,-2.038392,1.028223,2.379884,-0.927054,-0.155604,0.924169,0.925255,2.733481,1.015145,-0.767926,-3.827320,-3.155542,-0.439734,1.810095,1.021295,1.657145,-0.727603,-0.254092,1.169133,-1.008662,-1.932411,-1.053438,0.548321,-0.775278,-0.703259,-1.246567,0.058239,2.621367,-0.912691,-0.715110,-0.883699,1.660318,-2.670399,-1.243374,0.341603,-0.413621,-1.361476,1.560729,-1.409838,-0.958436,0.700707,-0.970159,-1.097722,-0.783886,-0.775900,2.148100,1.202156,-1.077570,-0.940244,0.538530,0.096820,-0.598834,-0.191359,1.379730,0.213528,-0.411468,-0.675531,-0.347392,-0.034143,-0.641644,0.625670,-0.447248,0.253629,dog_bark
1,1,-424.09818,109.340770,-52.919525,60.864750,0.245292,17.347328,2.095582,10.712965,-1.398613,12.310798,-11.208863,21.075323,-9.490279,13.526470,-0.239105,7.159000,-3.080976,11.367645,-6.302767,6.878115,-2.890210,8.761435,-2.330442,8.416637,-1.670082,1.277500,-4.419877,1.262629,-6.133947,0.863427,-1.031447,1.868078,-0.960069,0.036977,-0.015435,-0.071757,-1.598885,0.534893,-0.544687,0.446321,-0.303285,-1.293847,-1.160792,-0.293078,-0.452501,-0.119443,0.274645,-0.410808,-0.382534,-1.172179,0.210168,0.479906,0.454941,-1.349935,0.029096,-0.246033,-0.264071,-1.050533,-0.124349,0.232549,0.319697,-0.683531,0.023850,-0.544929,0.626820,0.327801,1.613409,-0.895195,0.162594,1.511366,1.158295,-0.728998,0.829437,-0.360532,-0.105445,-0.377364,0.815120,-0.145209,-0.641501,-0.713603,0.532129,-0.150462,0.077039,-0.190633,0.375228,-0.320643,-0.081479,-1.025114,0.107460,-0.470920,0.403968,-0.282803,0.408370,-0.320596,0.316777,0.162433,0.348257,-0.406998,-0.604030,-0.445381,-0.542503,-0.070650,0.065621,0.495111,0.310129,0.166576,-0.071223,-0.582726,0.067311,-0.266694,-0.232221,-0.016199,-0.424643,-0.052736,-0.217065,0.215534,-0.045850,0.572001,0.174419,-0.619190,-0.213965,-0.315515,0.183529,0.278578,0.129401,-0.470556,0.013517,-0.399829,children_playing
2,2,-458.79114,121.384190,-46.520660,52.008120,-0.000384,23.783813,8.749806,9.052491,-0.636421,10.987134,-11.845445,14.543654,-3.888519,11.987382,-1.036807,7.284361,-7.507326,4.264775,-3.103585,11.912483,0.334775,7.009369,-1.784793,2.674189,-3.427880,2.300696,-2.186260,-0.074099,-2.343275,1.640269,0.984424,-1.243447,-4.046101,-2.161726,-1.494173,-1.784590,-1.424619,2.076848,1.696297,-0.961409,1.439924,0.961798,-0.632394,-2.181090,0.698645,-1.526106,-1.120259,-1.304321,-0.767935,-1.127680,-0.019764,0.026335,-0.064974,-1.068498,-0.511537,-0.784570,0.684330,-0.001982,0.417576,-0.766691,0.497357,-0.459655,0.398695,-1.215428,0.262032,-0.311388,0.141161,0.165970,-0.197625,-0.896473,1.030774,-0.263212,0.194387,-0.032801,1.150658,0.882238,0.267670,0.298581,1.078961,-0.573184,0.363526,0.289110,-0.051320,-0.104241,0.413931,0.184103,0.040928,0.426101,0.615124,0.691548,-0.227779,-0.069813,0.664855,0.107534,0.030798,0.142822,-0.010631,-0.520481,-0.490568,-0.089468,-0.467474,-0.021346,-0.028859,0.071916,0.523125,0.123499,-0.211209,0.082366,-0.265598,0.033310,0.071

In [4]:
# Rename specific columns in the merged DataFrame

df.rename(columns={'0_x' : '0', '0_y' : 'class'}, inplace=True)

In [5]:
# Drop the 'Unnamed: 0' column from the DataFrame

df.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,class
0,-217.35526,70.22338,-130.385270,-53.282898,-21.199130,-22.677622,-10.855970,18.294254,6.652703,14.324025,-12.167682,2.276837,-17.779188,10.388951,-6.582836,-0.694457,-18.336023,1.994253,-5.143332,8.302400,-12.645056,-6.529732,4.617668,-2.179917,-6.662824,0.359711,-3.908410,4.775624,-6.384521,-5.379818,0.915978,6.970494,-0.248668,1.678219,-5.611182,-2.964347,3.149057,-1.693053,-0.616984,0.386005,1.476512,0.913438,-3.818676,-5.332023,-4.797818,-0.537269,-4.900532,0.346234,-4.434357,1.368662,-0.527099,-3.310836,-0.774530,-0.343457,-4.175725,-3.187561,-1.243311,-3.111319,0.200477,-2.898059,1.379016,2.010003,-0.557578,1.518796,-1.570530,-2.038392,1.028223,2.379884,-0.927054,-0.155604,0.924169,0.925255,2.733481,1.015145,-0.767926,-3.827320,-3.155542,-0.439734,1.810095,1.021295,1.657145,-0.727603,-0.254092,1.169133,-1.008662,-1.932411,-1.053438,0.548321,-0.775278,-0.703259,-1.246567,0.058239,2.621367,-0.912691,-0.715110,-0.883699,1.660318,-2.670399,-1.243374,0.341603,-0.413621,-1.361476,1.560729,-1.409838,-0.958436,0.700707,-0.970159,-1.097722,-0.783886,-0.775900,2.148100,1.202156,-1.077570,-0.940244,0.538530,0.096820,-0.598834,-0.191359,1.379730,0.213528,-0.411468,-0.675531,-0.347392,-0.034143,-0.641644,0.625670,-0.447248,0.253629,dog_bark
1,-424.09818,109.34077,-52.919525,60.864750,0.245292,17.347328,2.095582,10.712965,-1.398613,12.310798,-11.208863,21.075323,-9.490279,13.526470,-0.239105,7.159000,-3.080976,11.367645,-6.302767,6.878115,-2.890210,8.761435,-2.330442,8.416637,-1.670082,1.277500,-4.419877,1.262629,-6.133947,0.863427,-1.031447,1.868078,-0.960069,0.036977,-0.015435,-0.071757,-1.598885,0.534893,-0.544687,0.446321,-0.303285,-1.293847,-1.160792,-0.293078,-0.452501,-0.119443,0.274645,-0.410808,-0.382534,-1.172179,0.210168,0.479906,0.454941,-1.349935,0.029096,-0.246033,-0.264071,-1.050533,-0.124349,0.232549,0.319697,-0.683531,0.023850,-0.544929,0.626820,0.327801,1.613409,-0.895195,0.162594,1.511366,1.158295,-0.728998,0.829437,-0.360532,-0.105445,-0.377364,0.815120,-0.145209,-0.641501,-0.713603,0.532129,-0.150462,0.077039,-0.190633,0.375228,-0.320643,-0.081479,-1.025114,0.107460,-0.470920,0.403968,-0.282803,0.408370,-0.320596,0.316777,0.162433,0.348257,-0.406998,-0.604030,-0.445381,-0.542503,-0.070650,0.065621,0.495111,0.310129,0.166576,-0.071223,-0.582726,0.067311,-0.266694,-0.232221,-0.016199,-0.424643,-0.052736,-0.217065,0.215534,-0.045850,0.572001,0.174419,-0.619190,-0.213965,-0.315515,0.183529,0.278578,0.129401,-0.470556,0.013517,-0.399829,children_playing
2,-458.79114,121.38419,-46.520660,52.008120,-0.000384,23.783813,8.749806,9.052491,-0.636421,10.987134,-11.845445,14.543654,-3.888519,11.987382,-1.036807,7.284361,-7.507326,4.264775,-3.103585,11.912483,0.334775,7.009369,-1.784793,2.674189,-3.427880,2.300696,-2.186260,-0.074099,-2.343275,1.640269,0.984424,-1.243447,-4.046101,-2.161726,-1.494173,-1.784590,-1.424619,2.076848,1.696297,-0.961409,1.439924,0.961798,-0.632394,-2.181090,0.698645,-1.526106,-1.120259,-1.304321,-0.767935,-1.127680,-0.019764,0.026335,-0.064974,-1.068498,-0.511537,-0.784570,0.684330,-0.001982,0.417576,-0.766691,0.497357,-0.459655,0.398695,-1.215428,0.262032,-0.311388,0.141161,0.165970,-0.197625,-0.896473,1.030774,-0.263212,0.194387,-0.032801,1.150658,0.882238,0.267670,0.298581,1.078961,-0.573184,0.363526,0.289110,-0.051320,-0.104241,0.413931,0.184103,0.040928,0.426101,0.615124,0.691548,-0.227779,-0.069813,0.664855,0.107534,0.030798,0.142822,-0.010631,-0.520481,-0.490568,-0.089468,-0.467474,-0.021346,-0.028859,0.071916,0.523125,0.123499,-0.211209,0.082366,-0.265598,0.033310,0.071847,-0.115727,-0.489

In [6]:
# Save the DataFrame to a CSV file

df.to_csv('merged_df.csv', index=False)

In [9]:
# Display the count of unique values in the 'class' column

df['class'].value_counts()

class
dog_bark            1000
children_playing    1000
air_conditioner     1000
street_music        1000
engine_idling       1000
jackhammer          1000
drilling            1000
siren                929
car_horn             429
gun_shot             374
Name: count, dtype: int64

In [8]:
# Create a copy of the DataFrame for scaling

scaled_df = df.copy()

In [9]:
# Split the DataFrame into features (x1) and target variable (y1)

x1 = scaled_df.drop(columns=['class'])
y1 = scaled_df['class']

print(x1.shape)
print(y1.shape)

(8732, 128)
(8732,)


In [10]:
# Standardize the features in the DataFrame using StandardScaler

from sklearn.preprocessing import StandardScaler

def scaling(dataframe):
    scaler = StandardScaler()

    for col in dataframe.columns:
        dataframe[col] = scaler.fit_transform(dataframe[[col]])

scaling(x1)

In [11]:
# Encode the target variable using LabelEncoder from scikit-learn

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y1 = encoder.fit_transform(y1)

In [15]:
x1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.098916,-0.918738,-3.274024,-3.384654,-0.522299,-2.529374,-0.221542,0.603220,1.535350,0.555272,-0.479572,-0.720648,-1.219479,0.503790,-0.012595,-0.946853,-1.779119,-0.365733,0.016217,0.876904,-1.378535,-1.603790,1.569613,-0.908137,-0.758230,-0.279125,-0.294567,0.817048,-0.974286,-1.572213,0.629339,1.684273,0.183173,0.352386,-1.364543,-1.033254,1.310010,-0.271654,0.122490,0.055830,0.606605,0.482228,-1.122189,-1.873293,-1.450264,-0.032071,-1.735759,0.283564,-1.251606,0.478341,-0.014248,-1.059509,-0.148993,-0.034774,-1.791836,-1.294640,-0.370879,-1.213941,0.227784,-1.175503,0.767084,1.041063,-0.116561,1.092404,-0.631456,-0.964093,0.607410,0.957332,-0.265157,0.038140,0.577560,0.752611,2.013540,0.891596,-0.225479,-2.273203,-1.852839,-0.121306,1.309654,0.772281,1.155850,-0.260333,-0.023332,1.133097,-0.683769,-1.581206,-0.521697,0.687350,-0.407253,-0.400755,-0.867122,0.203303,2.665850,-0.645728,-0.460474,-0.738443,1.890953,-2.195322,-0.673378,0.546840,-0.152847,-1.487465,1.529907,-2.066731,-1.430826,0.665672,-1.125392,-1.140787,-0.793991,-1.046694,2.776024,1.545895,-1.537330,-1.322244,0.699819,0.078274,-0.897781,-0.346565,1.589503,0.369240,-0.568834,-0.830735,-0.676510,-0.083077,-1.085565,1.011265,-0.594243,0.529624
1,-1.495096,-0.103775,-0.805139,1.859902,0.841593,0.197031,0.740088,-0.075659,0.772967,0.322871,-0.371245,1.576391,-0.199814,0.951320,0.836280,0.187616,0.380928,1.054116,-0.182916,0.633840,0.310584,1.131292,0.278961,1.306985,0.318034,-0.076717,-0.417305,0.009594,-0.919153,-0.007189,0.218247,0.369684,0.023163,-0.109625,0.344846,-0.080892,-0.197082,0.256732,0.143060,0.070200,0.114796,-0.218794,-0.137347,0.027643,0.137204,0.125815,0.355855,0.001835,0.075401,-0.215394,0.219337,0.234963,0.359832,-0.488552,0.216181,0.020155,0.063992,-0.289027,0.074828,0.246789,0.304966,-0.139588,0.168728,-0.065457,0.551053,0.349266,0.887419,-0.263695,0.135382,0.667519,0.695615,-0.238362,0.788808,0.053316,0.143961,-0.004294,0.813001,0.069116,-0.259011,-0.271905,0.433671,0.042590,0.205459,0.029328,0.427774,-0.152895,0.218042,-0.451831,0.233893,-0.223070,0.465986,-0.110362,0.614576,-0.091532,0.472881,0.357028,0.556371,-0.113354,-0.186892,-0.110549,-0.272561,-0.060141,-0.173962,0.215269,0.081540,-0.000829,-0.070308,-0.528269,0.149386,-0.393105,-0.368728,-0.090943,-0.583467,-0.008094,-0.228091,0.215485,-0.152808,0.480270,0.269729,-0.692637,-0.293818,-0.325190,0.218069,0.455024,0.119862,-0.720356,0.086536,-0.552129
2,-1.762583,0.147135,-0.601203,1.452981,0.825968,0.635470,1.234153,-0.224349,0.845139,0.170070,-0.443166,0.778268,0.489289,0.731787,0.729537,0.205725,-0.245824,-0.021802,0.366545,1.492990,0.869012,0.817906,0.380318,0.106573,-0.060887,0.148936,0.118703,-0.297650,-0.085105,0.187546,0.643788,-0.431972,-0.670958,-0.728563,-0.106879,-0.644829,-0.141766,0.622426,0.780680,-0.265206,0.596495,0.497587,0.058443,-0.684607,0.557751,-0.405726,-0.207914,-0.330681,-0.050822,-0.203244,0.146489,0.080076,0.144662,-0.361664,-0.041999,-0.220559,0.485168,0.181580,0.330014,-0.207184,0.382468,-0.041457,0.352652,-0.441643,0.354741,-0.005517,0.182953,0.131933,0.002971,-0.241582,0.631314,0.040665,0.380326,0.253021,0.844439,0.824099,0.445451,0.356046,0.841835,-0.187391,0.325441,0.273307,0.116771,0.099455,0.458860,0.294399,0.311204,0.598861,0.602617,0.665945,-0.044265,0.085530,0.852318,0.309194,0.214209,0.336492,0.191323,-0.217740,-0.100557,0.186754,-0.202869,-0.005623,-0.281634,-0.291690,0.335470,-0.054581,-0.234610,0.262767,-0.219575,-0.008037,0.032989,-0.224657,-0.678766,0.490255,-0.037108,0.377005,-0.016006,0.004696,0.055160,0.524603,0.150729,-0.045839,-0.300351,-0

In [16]:
y1

array([3, 2, 2, ..., 1, 1, 1])

In [12]:
# Split the dataset into training and testing sets

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x1, y1, test_size=0.2, random_state=4)

In [18]:
# Print the shapes of training and testing sets

print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(6985, 128)
(1747, 128)
(6985,)
(1747,)


# Deep Learning

In [13]:
# Initializing the ANN 
model = Sequential()

In [14]:
# Adding input Layers
model.add(Dense(units=70, activation='relu', input_dim=128))

# Hidden layers with relu activation
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=200, activation='relu'))
model.add(Dropout(0.2))

# Output layer with softmax activation for multi-class classification
model.add(Dense(10, activation='softmax'))

In [44]:
early_stop = EarlyStopping(monitor='accuracy', patience=1, min_delta=0, mode='auto', verbose=0, restore_best_weights=True)

In [45]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [55]:
# Train the neural network model
model.fit(xtrain, ytrain, epochs=10, batch_size=32, callbacks=[early_stop])

Epoch 1/10
219/219 [==============================] - 1s 3ms/step - loss: 0.0371 - accuracy: 0.9891
Epoch 2/10
219/219 [==============================] - 1s 3ms/step - loss: 0.0303 - accuracy: 0.9908
Epoch 3/10
219/219 [==============================] - 1s 3ms/step - loss: 0.0236 - accuracy: 0.9913
Epoch 4/10
219/219 [==============================] - 1s 3ms/step - loss: 0.0205 - accuracy: 0.9931
Epoch 5/10
219/219 [==============================] - 1s 3ms/step - loss: 0.0151 - accuracy: 0.9948
Epoch 6/10
219/219 [==============================] - 1s 3ms/step - loss: 0.0244 - accuracy: 0.9911


In [56]:
# Evaluate the model on the testing set and print the test accuracy

test_accuracy = model.evaluate(xtest, ytest, verbose=0)
print(test_accuracy[1])

0.9124212861061096


In [57]:
# Make predictions on the training set
ytrain_pred = model.predict(xtrain)

# Convert the predictions to class labels
ytrain_pred_labels = np.argmax(ytrain_pred, axis=1)

# Create classification report for the training set
print(metrics.classification_report(ytrain, ytrain_pred_labels))

219/219 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       789
           1       1.00      1.00      1.00       346
           2       1.00      1.00      1.00       811
           3       1.00      1.00      1.00       811
           4       0.99      0.99      0.99       819
           5       1.00      1.00      1.00       794
           6       1.00      1.00      1.00       305
           7       0.99      0.99      0.99       811
           8       1.00      1.00      1.00       744
           9       1.00      1.00      1.00       755

    accuracy                           1.00      6985
   macro avg       1.00      1.00      1.00      6985
weighted avg       1.00      1.00      1.00      6985



In [58]:
# Make predictions on the testing set
ytest_pred = model.predict(xtest)

# Convert the predictions to class labels
ytest_pred_labels = np.argmax(ytest_pred, axis=1)

# Create classification report for the training set
print(metrics.classification_report(ytest, ytest_pred_labels))

55/55 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       211
           1       0.94      0.88      0.91        83
           2       0.81      0.87      0.84       189
           3       0.88      0.80      0.84       189
           4       0.93      0.94      0.94       181
           5       0.96      0.96      0.96       206
           6       0.84      0.84      0.84        69
           7       0.97      0.96      0.97       189
           8       0.94      0.95      0.94       185
           9       0.87      0.90      0.89       245

    accuracy                           0.91      1747
   macro avg       0.91      0.91      0.91      1747
weighted avg       0.91      0.91      0.91      1747



In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                9030      
                                                                 
 dense_1 (Dense)             (None, 128)               9088      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               25800     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2010      
                                                                 
Total params: 45928 (179.41 KB)
Trainable params: 45928 

### *Save the trained model using pickle*

In [60]:
# Save the model to a file named 'FModel.pkl' using pickle with the highest protocol

import pickle 

with open('FModel.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)          # protocol=pickle.HIGHEST_PROTOCOL parameter ensures the use of the highest available protocol for efficient serialization

In [61]:
#  Load the saved model from the file 'FModel.pkl' using pickle

with open('FModel.pkl', 'rb') as f:
    models = pickle.load(f)

In [62]:
# Save the model to an HDF5 file using Keras        -HDF5 (Hierarchical Data Format version 5) is a file format that allows storing and organizing large amounts of data.

models.save('FModel.h5')

In [151]:
# Import LabelEncoder from scikit-learn preprocessing module

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [152]:
class_labels = ['dog_bark', 'children_playing', 'car_horn', 'air_conditioner','street_music', 'gun_shot', 'siren', 'engine_idling', 'jackhammer','drilling']

In [153]:
#  Initialize and fit the LabelEncoder
encoder = LabelEncoder()
encoder.fit(class_labels)

LabelEncoder()

### *Testing*

In [155]:
# Function to extract MFCC(Mel-frequency cepstral coefficients) features from audio file

def extract_audio(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=128)  # Change n_mfcc to 128
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

    return mfccs_scaled_features

file = 'C:/Users/user/Desktop/Python/audio/144007-5-0-15.1.wav'

mfccs_scaled_features = extract_audio(file)


mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)

# Model prediction
predicted_probabilities = models.predict(mfccs_scaled_features)
predicted_label = np.argmax(predicted_probabilities)

# Round the predicted probabilities to handle numerical precision
rounded_probabilities = np.round(predicted_probabilities, decimals=15)

# Perform inverse transform on rounded predicted labels
prediction_class = encoder.inverse_transform(np.argmax(rounded_probabilities, axis=1))
print(prediction_class)


1/1 [==============================] - 0s 255ms/step
['siren']
